# Check for Intersecting Entities

Hi to all!
In data description it's stated "classify each element as **ONE** of the following...". Nevertheless, markup format with starts and ends allows intersected entities. So, maybe we need to check for shure existance of intersected entities in the markup? Let's check it.

Based on the great notebook by DAREK KŁECZEK [Feedback Prize EDA with displacy](https://www.kaggle.com/thedrcat/feedback-prize-eda-with-displacy)

In [ ]:
import pandas as pd
import os
from pathlib import Path
import spacy
from spacy import displacy

In [ ]:
path = Path('../input/feedback-prize-2021/train')

train = pd.read_csv('../input/feedback-prize-2021/train.csv')
train.head()

In [ ]:
def visualize_intersected_entities(example):
    ents = []
    for i, row in train[train['id'] == example].iterrows():
        ents.append({
                        'start': int(row['discourse_start']), 
                         'end': int(row['discourse_end']), 
                         'label': row['discourse_type']
                    })

    with open(path/f'{example}.txt', 'r') as file: data = file.read()
        
    ents.sort(key=lambda x: x['start'])
    
    intersections = []
    last_end = None
    last_label = None
    for e in ents:
        start = e['start']
        end = e['end']
        label = e['label']
        if last_end and last_end > start:
            intersections.append({
                'start': start,
                'end': last_end,
                'label': '_'.join([last_label, label])
            })
        if not last_end or end > last_end:
            last_end = end
            last_label = label

    doc2 = {
        "text": data,
        "ents": intersections,
        "title": example
    }

    if intersections:
        displacy.render(doc2, style="ent", manual=True, jupyter=True)
        print('\n')
        return True
    return False

In [ ]:
from tqdm import tqdm
examples = train['id'].values.tolist()

documents_with_intersections = 0
for ex in tqdm(examples):
    if visualize_intersected_entities(ex):
        documents_with_intersections += 1
print(f"documents_with_intersections: {documents_with_intersections}")

Well, nothing to worry about, we can do multiclass classification with peace of mind (: